In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import procore_formatter
import excel_creator
import metrics

In [2]:
raw = pd.read_csv('submittal_logs.csv')
df = procore_formatter.formatter(raw)

In [3]:
today = datetime.datetime.today()
date_folder = today.strftime("%Y-%m-%d")
directory =f'{date_folder} CIL Logs'
if not os.path.exists(directory):
    os.mkdir(directory)

In [4]:
def engineer(contractor):

    engineers = {
        'Ronny F' : ['Fred Geller Electric', 
                       'Corporate Electric',
                       'Societe International de Telecommunications Aeronautiques',
                       'Boingo Wireless',
                       'Schneider Electric',
                       'SITA',
                       'Iconics'],

        'Paul M' : ['B&G Electric',
                       'E-J Electric Installation Co.'],

        'Andy K' : ['FCS Group',
                       'Cassone Leasing, Inc.',
                       'JC Elite Construction',
                       'Lafayette Glass',
                       'Component Assembly Systems',
                       'Consolidated Carpet Trade Workroom',
                       'Garcia Marble and Tile',
                       'McKeon Door East'],

        'Nancy A' : ['Center Sheet Metal',
                        'PJ Mechanical LLC.',
                        'Olympic Plumbing And Heating Services Inc.',
                        'Rael Automatic Sprinkler'
                        ],

        'Alexandra H' : ['Schindler Elevator Corp.',
                            'Monpat Construction Inc.',
                            'Aero Bridge Works',
                            'John Bean Technologies Corporation',
                            'ILC Dover',
                            'W&W Glass, LLC.',
                            'Commodore Construction Corp.',
                            'Island International Enterprises',
                            'Yorie Tile & Terrazzo'
                            ],

        'Rebecca J' : ['Curtis Partition Corp.',
                         'Long Island Concrete',
                         'Post Road Iron Works Inc.',
                         'Long Island Concrete',
                         'Going Sign & Servicing Co. Inc.',
                         'Architectural Casework, Inc.'
                            ],
        'Andre F' : ['Peter Scalamandre & Sons Inc',
                     'Owen Steel Company, Inc.',
                     'Falco Construction Corp.'
                  ]
    }
    
    for key, value in engineers.items():
        if contractor == value:
            return key
        
        elif isinstance(value, list) and contractor in value:
            return key
        
    return 'Miscellaneous'

In [5]:
def trade(contractor):

    trade = {
        'Fred Geller Electric' : 'Low Voltage', 
        'Corporate Electric' : 'Fire Alarm Public Address',
        'Societe International de Telecommunications Aeronautiques' : 'SITA',
        'Boingo Wireless' : 'BOINGO',
        'Schneider Electric' : 'BMS',
        'SITA' : 'SITA',
        'Iconics' : 'Iconics',
        'B&G Electric' : 'ELEC FO',
        'E-J Electric Installation Co.' : 'ELEC PD',
        'FCS Group' : 'Painting',
        'Cassone Leasing, Inc.' : 'CCF Connector',
        'JC Elite Construction' : 'CCD Connector',
        'Lafayette Glass' : 'Ornamental Metal',
        'Component Assembly Systems' : 'Carpenter',
        'Consolidated Carpet Trade Workroom' : 'Carpet',
        'Garcia Marble and Tile' : 'Tile',
        'McKeon Door East' : 'Coiling Door',
        'Center Sheet Metal' : 'HVAC Air',
        'PJ Mechanical LLC.' : 'HVAC Wet',
        'Olympic Plumbing And Heating Services Inc.' : 'Plumbing',
        'Rael Automatic Sprinkler' : 'Sprinkler',
        'Schindler Elevator Corp.' : 'Vert Transport',
        'Monpat Construction Inc.' : 'Roofing',
        'Aero Bridge Works' : 'Aero',
        'John Bean Technologies Corporation' : 'Jetbridge',
        'ILC Dover' : 'Flood Barrier',
        'W&W Glass, LLC.' : 'Curtainwall',
        'Commodore Construction Corp.' : 'Concrete',
        'Island International Enterprises' : 'SOFP',
        'Yorie Tile & Terrazzo' : 'Terrazo',
        'Curtis Partition Corp.' : 'Toilet Accessories',
        'Long Island Concrete' : 'Masonry',
        'Post Road Iron Works Inc.' : 'Misc Metal',
        'Long Island Concrete' : 'Masonry',
        'Going Sign & Servicing Co. Inc.' : 'Signage',
        'Architectural Casework, Inc.' : 'Furniture',
        'Peter Scalamandre & Sons Inc' : 'Foundations',
        'Owen Steel Company, Inc.' : 'Steel',
        'Falco Construction Corp.' : 'Piles',
        'Delta CMT' : 'Delta CMT',
        'Skyline Steel LLC' : 'Skyline Steel',
        'Skyline Steel Corp.' : 'Skyline Steel 1',
        'STV S&P, JV' : 'STV', 
        'NYCO Environmental' : 'NYCO',
        'Foundation Engineering Services, D.P.C.' : 'Foundations Eng.',
        'Infax' : 'Restroom Monitoring',
        'Turner Construction' : 'CMT', 
        'nan' : 'Misc.', 
        'Kusch + Co' : 'Seating', 
        'Image Manufacturing Group' : 'Kiosks', 
        'Office Images' : 'Office Furnishing', 
        'Flockheart' : 'Holdroom Seats',
        'Nanolumens' : 'Wayfinding', 
        'Contractor' : 'Misc. 2'
    }
    
    for key, value in trade.items():
        if contractor == key:
            return value
        
    return 'Misc. '

In [6]:
df['Responsible Contractor'] = df['Responsible Contractor'].str.replace('/', ' ')
contractors = df['Responsible Contractor'].unique()

In [7]:
closeout = ['As-Built',
           'Attic Stock',
           'Certificate of Substantial Completion',
           'Certificate of Final Completion',
           'Closeouts',
           'O&M Manual',
           'Product Manual',
           'Record Documents',
           'Spare Parts List',
           'Training Material',
           'Warranty / Guarantee']

types = df['Description'].unique()
not_closeout = [x for x in types if x not in closeout]

default_font = 'Calibri'

In [8]:
i = 1
data = df
name = "Master"

name_directory = directory + f'/{name}'
if not os.path.exists(name_directory):
    os.mkdir(name_directory)

writer = pd.ExcelWriter(f'{name_directory}/Master CIL Log {date_folder}.xlsx', 
                    engine = 'xlsxwriter',
                    date_format = 'mm-dd-yyyy', 
                    datetime_format='mm-dd-yyyy')
workbook = writer.book

data.to_excel(writer,
             sheet_name = 'Submittals Master Log',
             index = False,
             startrow = 8)
submittal_log = writer.sheets['Submittals Master Log']
excel_creator.generate_worksheet(data, name, contractors[i], workbook, submittal_log)

number_total_format = workbook.add_format({
'bold' : True,
'font_color' : 'black', 
'font_size' : 11,
'font_name' : default_font, 
'align' : 'center',
'valign' : 'vcenter', 
'border' : 1,
'num_format' : '#,##0'
})

percent_format = workbook.add_format({
    'bold' : False,
    'font_color' : 'black', 
    'font_size' : 11,
    'font_name' : default_font, 
    'align' : 'center',
    'valign' : 'vcenter', 
    'border' : 1,
    'num_format' : '0.00%'
})

submittal_log.write('J1', 'North #')
submittal_log.write('K1', 'North %')
submittal_log.write('L1', 'South #')
submittal_log.write('M1', 'South %')
submittal_log.write('I2', 'Total')
submittal_log.write('I3', 'Open')
submittal_log.write('I4', 'Closeout')
submittal_log.write('I5', 'Design Team')
submittal_log.write('I6', 'R&R')
submittal_log.write('I7', 'R&R LEED')
submittal_log.write('I8', 'Closed')

north_metrics = metrics.metrics(data, ['N', 'N/S'])
total = 0
for key in north_metrics:
    total = total + north_metrics[key][1]

submittal_log.write('J2', total, number_total_format)
if total == 0:
    total = 1 

row = 2
for key in north_metrics:
    submittal_log.write(row, 9, north_metrics[key][1], number_total_format)
    submittal_log.write(row, 10, north_metrics[key][1]/total, percent_format)
    row = row + 1


south_metrics = metrics.metrics(data, ['S'])
total = 0
for key in south_metrics:
    total = total + south_metrics[key][1]


submittal_log.write('L2', total, number_total_format)
if total == 0:
    total = 1  

row = 2
for key in south_metrics:
    submittal_log.write(row, 11, south_metrics[key][1], number_total_format)
    submittal_log.write(row, 12, south_metrics[key][1]/total, percent_format)
    row = row + 1

In [9]:
for i in range(len(contractors)):
    
    data = df[df['Responsible Contractor'] == contractors[i]]
    name = engineer(contractors[i])
    trades = trade(contractors[i])
        
    workbook = writer.book
    
    data.to_excel(writer,
                 sheet_name = f'{trades}',
                 index = False,
                 startrow = 8)
    submittal_log = writer.sheets[f'{trades}']
    excel_creator.generate_worksheet(data, trades, contractors[i], workbook, submittal_log)
    
    number_total_format = workbook.add_format({
    'bold' : True,
    'font_color' : 'black', 
    'font_size' : 11,
    'font_name' : default_font, 
    'align' : 'center',
    'valign' : 'vcenter', 
    'border' : 1,
    'num_format' : '#,##0'
    })

    percent_format = workbook.add_format({
        'bold' : False,
        'font_color' : 'black', 
        'font_size' : 11,
        'font_name' : default_font, 
        'align' : 'center',
        'valign' : 'vcenter', 
        'border' : 1,
        'num_format' : '0.00%'
    })
    
    submittal_log.write('J1', 'North #')
    submittal_log.write('K1', 'North %')
    submittal_log.write('L1', 'South #')
    submittal_log.write('M1', 'South %')
    submittal_log.write('I2', 'Total')
    submittal_log.write('I3', 'Open')
    submittal_log.write('I4', 'Closeout')
    submittal_log.write('I5', 'Design Team')
    submittal_log.write('I6', 'R&R')
    submittal_log.write('I7', 'R&R LEED')
    submittal_log.write('I8', 'Closed')
    
    north_metrics = metrics.metrics(data, ['N', 'N/S'])
    total = 0
    for key in north_metrics:
        total = total + north_metrics[key][1]
    
    submittal_log.write('J2', total, number_total_format)
    if total == 0:
        total = 1 
        
    row = 2
    for key in north_metrics:
        submittal_log.write(row, 9, north_metrics[key][1], number_total_format)
        submittal_log.write(row, 10, north_metrics[key][1]/total, percent_format)
        row = row + 1
        
        
    south_metrics = metrics.metrics(data, ['S'])
    total = 0
    for key in south_metrics:
        total = total + south_metrics[key][1]

        
    submittal_log.write('L2', total, number_total_format)
    if total == 0:
        total = 1  
        
    row = 2
    for key in south_metrics:
        submittal_log.write(row, 11, south_metrics[key][1], number_total_format)
        submittal_log.write(row, 12, south_metrics[key][1]/total, percent_format)
        row = row + 1
 
    print(contractors[i])

C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Type'] = df['Description']
C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Location'] = df['Phase']
C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

Aero Bridge Works
Schindler Elevator Corp.


C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Location'] = df['Phase']
C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Type'] = df['Description']
C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

Island International Enterprises
Monpat Construction Inc.


C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Type'] = df['Description']
C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Location'] = df['Phase']
C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

...

C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Location'] = df['Phase']
C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Type'] = df['Description']
C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

Falco Construction Corp.
Skyline Steel LLC


C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Type'] = df['Description']
C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Location'] = df['Phase']
C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

STV S&P, JV
Skyline Steel Corp.


C:\Users\kfong\Programs\Submittal Log\Personal Submittal Log V2\metrics.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Location'] = df['Phase']


In [10]:
    writer.close()
    writer.handles = None